EEG Spike-Wave visualization tool
Copyright (C) 2023  Leonardo Cañete-Sifuentes, Azul Salmerón Ruiz, Lizbeth Becerril Belio 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

Repository: https://github.com/LeonardoC/SpikeWave-Visualization

In [18]:
import pandas as pd
from linelength_event_detector.lleventdetector import *
from linelength_event_detector.lltransform import *
import plotly.express as px
import plotly.graph_objects as go
from process_data import *

In [19]:
data = pd.read_csv(r'C:/Users/L03109567/Desktop/LNHV- Análisis/E3,4,5,6/LB 1/LB 1.csv', header = None)
data = data.to_numpy()
data = data.transpose()

In [20]:
sfx = 500 # Sampling frequency
mel = 500 # Minimum event length (milliseconds)
prc = 99
llw = 1
events = detect_events(data, 0, sfx, mel, llw, prc) 
subject_windows, subject_channels = split_events(events)

In [21]:
def detect_hill(x):
    a, b = x
    return a <= 0 and b > 0
test_list = [4, 7, 3, 2, 9, 2, 1]
diffs = np.diff(test_list)
diffs

array([ 3, -4, -1,  7, -7, -1])

In [22]:
hills = list(map(detect_hill, zip(diffs[:-1], diffs[1:])))

In [23]:
sum(hills)

1